# Using Aviation Edge and OpenSky APIs for historical data analysis

## Phase 1: Data Collection

### Real-Time Flight Data

In [4]:
# Importing modules

import requests
import pandas as pd
import nest_asyncio
import asyncio
from datetime import datetime, timedelta
import time
import json
import random

In [5]:
# Setting up OpenSky

from python_opensky import OpenSky
nest_asyncio.apply()

class AirCanadaDataCollector:
    def __init__(self):
        self.flight_data = []
        self.historical_data = pd.DataFrame()   #Dataframe for storing Air Canada Flights
    
    async def collect_realtime_flights(self):
        """Collects current Air Canada flight data"""
        async with OpenSky() as opensky:
            
            # Getting most recent flight data
            states = await opensky.get_states()
            
            ac_flights = []

            # Filtering to get Air Canada flight data only
            for state in states.states:
                if state.callsign and state.callsign.strip().startswith('ACA'):
                    flight_info = {
                        'timestamp': datetime.now(),
                        'callsign': state.callsign.strip(),
                        'icao24': state.icao24,
                        'longitude': state.longitude,
                        'latitude': state.latitude,
                        'altitude': state.barometric_altitude,
                        'velocity': state.velocity,
                        'heading': state.true_track,
                        'vertical_rate': state.vertical_rate,
                        'on_ground': state.on_ground
                    }

                    # Adds the current Air Canada flight details to the list
                    ac_flights.append(flight_info)
            
            return pd.DataFrame(ac_flights)
    
    async def collect_historical_flights(self, start_time, end_time):
        """Collects historical flight data for a specific time range"""
        async with OpenSky() as opensky:
            # OpenSky provides historical data access
            flights = await opensky.get_flights_by_aircraft(
                begin=int(start_time.timestamp()),
                end=int(end_time.timestamp())
            )
            return flights

# Initializes collector object
collector = AirCanadaDataCollector()

# Collectes current flights
current_flights = await collector.collect_realtime_flights()
print(f"Collected {len(current_flights)} current Air Canada flights")
print(current_flights)

Collected 74 current Air Canada flights
                    timestamp callsign  icao24  longitude  latitude  altitude  \
0  2025-06-30 18:23:07.094181  ACA1092  c07e32  -117.1822   32.7307       NaN   
1  2025-06-30 18:23:07.094181   ACA308  c0104f  -117.5138   49.5309  11887.20   
2  2025-06-30 18:23:07.094181    ACA23  c0103a  -124.3142   49.9103   6659.88   
3  2025-06-30 18:23:07.094181   ACA323  c0103e  -108.8140   51.5591  12192.00   
4  2025-06-30 18:23:07.094181     ACA7  c01049  -150.0134   60.6552  10972.80   
..                        ...      ...     ...        ...       ...       ...   
69 2025-06-30 18:23:07.096181   ACA840  c04fdd   -78.9389   43.7099   5295.90   
70 2025-06-30 18:23:07.096181   ACA267  c005c5   -82.5786   45.4632  10363.20   
71 2025-06-30 18:23:07.096181   ACA119  c00821   -81.9888   45.2874   9753.60   
72 2025-06-30 18:23:07.096181  ACA1049  c006ec   -87.9042   41.9704       NaN   
73 2025-06-30 18:23:07.096181  ACA1058  c00757  -122.5047   47.7652  

### Air Canada Route and Schedule Data

In [7]:
# Creating a class processing airport data and identifying high traffic routes

class AirCanadaRoute:
    def __init__(self):
        self.major_hubs = ["YUL", "YYZ", "YYC", "YVR"]  # Major Hubs: Montreal, Toronto, Calgary & Vancouver
        self.route_data = []

    def get_airport_data(self):
        """Gets airport traffic data from reliable sources (yyc.com, admtl.com, internationalairportreview.com and torontopearson.com)"""
        
        airport_data = {
            "YUL" : {"name": "Montreal Trudeau", "annual_passengers": "22400000"},
            "YYZ" : {"name": "Toronto Pearson", "annual_passengers": "46800000"},
            "YYC" : {"name": "Calgary International", "annual_passengers": "18900000"},
            "YVR" : {"name": "Vancouver International", "annual_passengers": "26200000"}
        }
        return airport_data

    def identify_high_traffic_routes(self, flight_data):
        """Identifies highest traffic routes for analysis"""
        route_frequency = {}
        
        for _, flight in flight_data.iterrows():

            # Extracts route information from callsign patterns
            callsign = flight["callsign"]
            
            # Air Canada uses specific callsign patterns for different routes
            if callsign.startswith('ACA'):
                flight_number = callsign[3:]

                # Key: flight_number, value: route_frequency
                route_frequency[flight_number] = route_frequency.get(flight_number, 0) + 1
        return dict(sorted(route_frequency.items(), key=lambda x:x[1], reverse=True))

In [8]:
#  Creating AirCanadaRoute objects

route_collector = AirCanadaRoute()
airport_data = route_collector.get_airport_data()
high_traffic_routes = route_collector.identify_high_traffic_routes(current_flights)

In [9]:
print(high_traffic_routes)

{'1092': 1, '308': 1, '23': 1, '323': 1, '7': 1, '9XC': 1, '488': 1, '788': 1, '743': 1, '487': 1, '576': 1, '19': 1, '025': 1, '907': 1, '33': 1, '146': 1, '983': 1, '112': 1, '585': 1, '1075': 1, '733': 1, '815': 1, '592': 1, '506': 1, '305': 1, '581': 1, '216': 1, '567': 1, '336': 1, '1282': 1, '7249': 1, '895': 1, '344': 1, '617': 1, '935': 1, '461': 1, '615': 1, '1066': 1, '217': 1, '2114': 1, '266': 1, '668': 1, '091': 1, '326': 1, '1176': 1, '6': 1, '556': 1, '990': 1, '791': 1, '030': 1, '599': 1, '164': 1, '778': 1, '306': 1, '356': 1, '897': 1, '891': 1, '460': 1, '115': 1, '1308': 1, '489': 1, '758': 1, '555': 1, '108': 1, '171': 1, '113': 1, '264': 1, '909': 1, '548': 1, '840': 1, '267': 1, '119': 1, '1049': 1, '1058': 1}


### External Data Collection (Weather, Holidays & Seasonal Factors)

In [16]:
class ExternalDataCollector:
    def __init__(self):
        # Processes weather data from 'api.weather.gc.ca' and public holidays for later use
        
        self.weather_api = "https://api.weather.gc.ca"
        self.holidays_2025 = [
            "2025-01-01",  # New Year's Day
            "2025-04-18",  # Good Friday
            "2025-05-19",  # Victoria Day
            "2025-07-01",  # Canada Day
            "2025-09-01",  # Labour Day
            "2025-10-13",  # Thanksgiving
            "2025-11-11",  # Remembrance Day
            "2025-12-25",  # Christmas Day
            "2025-12-26"   # Boxing Day
        ]
    
    def get_weather_data(self, airport_code, date):
        """Gets weather data for specific airports for specific dates"""
        
        weather_data = {
            "airport": airport_code,
            "date": date,
            "temperature": None,
            "percipitation": None,
            "visibility": None,
            "wind_speed": None,
            "conditions": None
        }
        return weather_data


    def is_holiday(self, date):
        """Checks if the specific date is a holiday or not"""
        
        date_str = date.strftime("%Y-%m-%d")
        return date_str in self.holidays_2025

    def get_seasonal_factors(self, date):
        """Gets seasonal adjustment factors"""
        
        month = date.month
        seasons = {
            "winter_months": [12, 1, 2],
            "spring_months": [3, 4, 5],
            "summer_months": [6, 7, 8],
            "fall_months": [9, 10, 11]
        }

        for season, months in seasons.items():
            if month in months:
                return season
            else:
                return "Unknown"

In [18]:
# Creating ExternalDataCollector object

external_collector = ExternalDataCollector()

### Passenger Booking Pattern Simulator

In [34]:
# Realistic Passenger Booking Simulation since I don't have access to real passenger data immediately

class PassengerBookingSimulator:
    def __init__(self):
        # Passenger booking features that affects no-show probability
        
        self.fare_class = ["Basic", "Standard", "Flex", "Premium", "Business"]
        self.advance_purchase_window = [1, 7, 14, 30, 60, 90]  # Time in days
        self.passenger_types = ["Business", "Leisure", "VFR"]

    def generate_booking_data(self, flight_info, num_passengers = 180):
        """Generates realistic passenger booking data for a flight based on available research"""
        
        passengers = []

        for i in range(0, num_passengers):
            # Generates passenger features that research shows affect no-show rates

            advance_purchase = random.choice(self.advance_purchase_window)
            fare_class = random.choice(self.fare_class)
            passenger_type = random.choice(self.passenger_types)

            # Factors that co-relate with no-show probability according to research (https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=d3572abdaf3d12bd8e85471b1b217c495f7662d9)
            base_no_show_prob = 0.12  # According to researches, the industry average is between 5%-15% with 12% being a commonly cited average

            if advance_purchase <= 7:
                base_no_show_prob *= 0.7  # Last-minute bookings are less likely to be a no-show. Based on 30% reduction for last minute booking is based on general industry logic

            if fare_class in ["Premium", "Business"]:
                base_no_show_prob *= 0.5  # Premium and Business bookings are less likely to be a no-show. More of an assumption so 50/50 chances

            if passenger_type == "Business":
                base_no_show_prob *= 0.6  # Passengers travelling for business are less likely to be a no-show. 40% reduction for business travelers is based on general industry logic

            passenger = {
                "passenger_id": f"PAX_{flight_info['callsign']}_{i:03d}",
                "flight_callsign": flight_info["callsign"],
                "booking_data": datetime.now() - timedelta(days= advance_purchase),
                "advance_purchase_days": advance_purchase,
                "fare_class": fare_class,
                "passenger_type": passenger_type,
                "predicted_no_show_prob": min(base_no_show_prob, 0.15),  # Caps at 15%
                "actual_no_show": random.random() > base_no_show_prob  # Returns boolean
            }
            passengers.append(passenger)

        return pd.DataFrame(passengers)

In [36]:
# Creating PassengerBookingSimulation object

passenger_simulation = PassengerBookingSimulator()

### Building a data collection pipeline

In [39]:
# AirCanadaPipeline class collects real-time flight details, analyzes the routes, collects external data for each flight and predicts the no show rate

In [47]:
class AirCanadaPipeline:

    def __init__(self):
        
        # Initializing all the objects
        self.flight_collector = AirCanadaDataCollector()
        self.route_collector = AirCanadaRoute()
        self.external_data_collector = ExternalDataCollector()
        self.passenger_simulator = PassengerBookingSimulator()

    async def run_complete_collection(self):
        """Runs the complete data collection"""
        print("Starting Air Canada Data Collection...")

        # Collects real-time Air Canada flight data
        print("Collecting real-time flight data...")
        current_flights = await self.flight_collector.collect_realtime_flights()
        
        # Analyze the routes for the flights
        print("Analyzing flight route patterns...")
        high_traffic_routes = self.route_collector.identify_high_traffic_routes(current_flights)
        
        # Collect external data for each flights
        print("Collecting external factors...")
        tuned_flights = []

        # Iterates through each flight
        for _, flight, in current_flights.iterrows():
            
            # Adds weather data
            weather = self.external_data_collector.get_weather_data("YYZ", datetime.now())

            # Adds holiday data
            is_holiday = self.external_data_collector.is_holiday(datetime.now())

            # Adds seasonal factor
            season = self.external_data_collector.get_seasonal_factors(datetime.now())

            # Runs passenger booking simulation for this flight
            passengers = self.passenger_simulator.generate_booking_data(flight)

            tuned_flight = {
                **flight.to_dict(),  # Converts the flight data to dictionary representation
                "weather_conditions": weather,
                "is_holiday": is_holiday,
                "season": season,
                "passenger_count": len(passengers),
                "predicted_no_shows": passengers["predicted_no_show_prob"].sum(),   # Predicted number of no-show passengers
                "historical_no_show_rate": passengers["predicted_no_show_prob"].mean()  # Average no-show rate for flight
            }

            tuned_flights.append(tuned_flight)  # Adds the simulation data to the tuned_flights list

        tuned_df = pd.DataFrame(tuned_flights)  # Creates DataFrame for the tuned_flights

        print(f"Complete! Collected data for {len(tuned_df)} flights.")
        print(f"Average predicted no-show rate: {tuned_df['historical_no_show_rate'].mean():.3f}")

        return tuned_df, high_traffic_routes

In [49]:
# Runs the Pipeline

pipeline = AirCanadaPipeline()
flight_data, route_analysis = await pipeline.run_complete_collection()
        
# Displays the output of pipeline

print("\n*** Air Canada Flight Analysis ***")
print(flight_data[['callsign', 'passenger_count', 'predicted_no_shows', 'historical_no_show_rate']].head())

print(f"\n*** Top 5 Air Canada Routes by Frequency: ***")
for route, frequency in list(route_analysis.items())[:5]:
    print(f"AC{route}: {frequency} flights observed.")
    

Starting Air Canada Data Collection...
Analyzing flight route patterns...
Complete! Collected data for 89 flights.
Average predicted no-show rate: 0.074

*** Air Canada Flight Analysis ***
  callsign  passenger_count  predicted_no_shows  historical_no_show_rate
0   ACA512              180             12.8544                 0.071413
1  ACA1092              180             13.0308                 0.072393
2   ACA308              180             13.2384                 0.073547
3    ACA23              180             12.8220                 0.071233
4   ACA323              180             12.9420                 0.071900

*** Top 5 Air Canada Routes by Frequency: ***
AC512: 1 flights observed.
AC1092: 1 flights observed.
AC308: 1 flights observed.
AC23: 1 flights observed.
AC323: 1 flights observed.
